In [13]:
from os import listdir, path
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import argparse, os, cv2, traceback, subprocess
from tqdm import tqdm
from glob import glob
import audio
from hparams import hparams as hp
import torch

import face_detection

## Define variables

In [36]:
ngpu = 1
batch_size = 32
data_root = "c:/Users/ioann/Desktop/projects/Github/Wav2Lip/data/wav2lip_homework/mvlrs_v1/main"
preprocessed_root = "c:/Users/ioann/Desktop/projects/Github/Wav2Lip/data/preprocess"


In [37]:
# Define the parser
parser = argparse.ArgumentParser()

# Add arguments
parser.add_argument('--ngpu', help='Number of GPUs across which to run in parallel', default=1, type=int)
parser.add_argument('--batch_size', help='Single GPU face detection batch size', default=32, type=int)
parser.add_argument("--data_root", help="Root folder of the LRS2 dataset", required=True)
parser.add_argument("--preprocessed_root", help="Root folder of the preprocessed dataset", required=True)

# Parse the arguments with the values
args = parser.parse_args(args=["--ngpu", str(ngpu), 
                               "--batch_size", str(batch_size), 
                               "--data_root", data_root, 
                               "--preprocessed_root", preprocessed_root])

In [38]:
fa = [face_detection.FaceAlignment(face_detection.LandmarksType._2D, flip_input=False, 
									device='cuda:{}'.format(id)) for id in range(ngpu)]

template = 'ffmpeg -loglevel panic -y -i {} -strict -2 {}'

## Functions

In [39]:
def process_video_file(vfile, args, gpu_id):
	video_stream = cv2.VideoCapture(vfile)
	
	frames = []
	while 1:
		still_reading, frame = video_stream.read()
		if not still_reading:
			video_stream.release()
			break
		frames.append(frame)
	
	vidname = os.path.basename(vfile).split('.')[0]
	dirname = vfile.split('/')[-2]

	fulldir = path.join(args.preprocessed_root, dirname, vidname)
	os.makedirs(fulldir, exist_ok=True)

	batches = [frames[i:i + args.batch_size] for i in range(0, len(frames), args.batch_size)]

	i = -1
	for fb in batches:
		preds = fa[gpu_id].get_detections_for_batch(np.asarray(fb))

		for j, f in enumerate(preds):
			i += 1
			if f is None:
				continue

			x1, y1, x2, y2 = f
			cv2.imwrite(path.join(fulldir, '{}.jpg'.format(i)), fb[j][y1:y2, x1:x2])

In [40]:
def process_audio_file(vfile, args):
	vidname = os.path.basename(vfile).split('.')[0]
	dirname = vfile.split('/')[-2]

	fulldir = path.join(args.preprocessed_root, dirname, vidname)
	os.makedirs(fulldir, exist_ok=True)

	wavpath = path.join(fulldir, 'audio.wav')

	command = template.format(vfile, wavpath)
	subprocess.call(command, shell=True)

In [41]:
def mp_handler(job):
	vfile, args, gpu_id = job
	try:
		process_video_file(vfile, args, gpu_id)
	except KeyboardInterrupt:
		exit(0)
	except:
		traceback.print_exc()

## Main

In [42]:
torch.cuda.device_count()

1

In [43]:
print('Started processing for {} with {} GPUs'.format(args.data_root, args.ngpu))

Started processing for c:/Users/ioann/Desktop/projects/Github/Wav2Lip/data/wav2lip_homework/mvlrs_v1/main with 1 GPUs


In [44]:
filelist = glob(path.join(args.data_root, '*/*.mp4'))[:10]

In [45]:
jobs = [(vfile, args, i%args.ngpu) for i, vfile in enumerate(filelist)]

In [49]:
p = ThreadPoolExecutor(args.ngpu)
futures = [p.submit(mp_handler, j) for j in jobs]

In [50]:
_ = [r.result() for r in tqdm(as_completed(futures), total=len(futures))]

100%|██████████| 10/10 [00:24<00:00,  2.46s/it]


In [52]:
filelist[0]

'c:/Users/ioann/Desktop/projects/Github/Wav2Lip/data/wav2lip_homework/mvlrs_v1/main\\5535415699068794046\\00001.mp4'

In [51]:
print('Dumping audios...')

for vfile in tqdm(filelist):
    try:
        process_audio_file(vfile, args)
    except KeyboardInterrupt:
        exit(0)
    except:
        traceback.print_exc()
        continue

Dumping audios...


100%|██████████| 10/10 [00:01<00:00,  9.53it/s]
